In [48]:
import random
import numpy as np  
import scipy
from math import sqrt

In [7]:
alpha = 0.05
n = 1000000

def cyclic_right_shift(num, k, n):
    return (num >> k) | (num << (n - k))

def cyclic_left_shift(num, k, n):
    return (num << k) | (num >> (n - k))

def bits_to_bytes(a):
    i = 0
    res = []
    for i in range(0, len(a) - 1, 8):
        x = a[i:i + 8]
        y = 0
        for j in range(len(x)):
            y += 2 ** j * x[j]
        res.append(y)   
    return res

Тести:

In [8]:
def uniformity_test(Y):
    R = {}
    n = len(Y) / 256
    X2 = 0 
    for i in range(256):
        R[i] = 0
    for y in Y:
        R[y] += 1
        X2 += (R[y] - n) ** 2 / n
    X2_l = sqrt(2 * 255) * scipy.stats.norm.ppf(1 - alpha) + 255
    if X2 <= X2_l:
        return 0
    return 1

def independence_test(Y):
    R = {}
    n = round(len(Y) / 2)
    i = len(Y) - 1
    for i in range(256):
        for j in range(256):
            R[i, j] = 0
    while 2 * i >= 1:
        R[Y[2*i - 1], Y[2 * i]] += 1
        i = i - 1
    X2 = 0
    for i in range(256):
        v_i = 0
        for j in range(256):
            a_j = 0
            if R[i,j] != 0:
                for k in R.keys():
                    if k[0] == i:
                        v_i += R[k]
                for k in R.keys():
                    if k[1] == j:
                        a_j += R[k]
                X2 += R[i,j] / v_i * a_j
    X2 = n * (X2 - 1)
    X2_l = sqrt(2 * 255 ** 2) * scipy.stats.norm.ppf(1 - alpha) + 255 ** 2
    if X2 <= X2_l:
        return 0
    return 1        

def homogeneity_test(Y):
    R = {}
    r = 100
    m_len = round(len(Y) / r)
    n = m_len * r
    for i in range(256):
        R[i] = 0
    for y in Y:
        R[y] += 1
    X2 = 0
    for i in range(256):
        if R[i] != 0:
            t = 0
            for j in range(1, r):
                temp = Y[(r-1) * m_len : r * m_len]
                for s in temp: 
                    if s == i: 
                        t += 1
                X2 += t ** 2 / R[i] * m_len 
    X2 = n * (X2 - 1)
    X2_l = sqrt(2 * 255 * (r - 1)) * scipy.stats.norm.ppf(1 - alpha) +  255 * (r - 1)
    if X2 <= X2_l:
        return 0
    return 1           

def test(Y):
    if uniformity_test(Y):
        print("Всі байти рівноімовірні.")
    else: print("1. :(")
    if independence_test(Y):
        print("Всі байти незалежні.")
    else: print("2. :(")
    if homogeneity_test(Y):
        print("Послідовність однорідна")
    else: print("3. :(")

Генератори: 

In [49]:
#Вбудований:
def integrated_gen(n):
    seq = np.random.randint(256, size=(n))
    return seq

print("Вбудований генератор:")
test(integrated_gen(n))

Вбудований генератор:
Всі байти рівноімовірні.
Всі байти незалежні.
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\masha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\IPython\core\interactiveshell.py", line 3378, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\masha\AppData\Local\Temp\ipykernel_14156\329776554.py", line 7, in <module>
    test(integrated_gen(n))
  File "C:\Users\masha\AppData\Local\Temp\ipykernel_14156\1231616181.py", line 76, in test
    if homogeneity_test(Y):
  File "C:\Users\masha\AppData\Local\Temp\ipykernel_14156\1231616181.py", line -1, in homogeneity_test
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\masha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\IPython\core\interactiveshell.py", line 1997, in showtraceback
    stb = self.InteractiveTB.s

In [ ]:
#Лемера:
def lemer_gen(state, t):
    m = 2 ** 32
    a = 2 ** 16 + 1
    c = 119 
    seq = []
    i = 0
    x_i = 0
    x_0 = random.randint(1, m)
    while i != t:
        x_i = (a * x_0 + c) % m
        x_0 = x_i
        i += 1
        if state == "low":
            seq.append(x_i % (2**8))
        elif "high": 
            seq.append(x_i >> 24)
    return seq 


print("Генератор Лемера low")
test(lemer_gen("low", n))

print("Генератор Лемера high")
test(lemer_gen("high", n))

Генератор Лемера low
Всі байти рівноімовірні.
Всі байти незалежні.
Послідовність однорідна
Генератор Лемера high
Всі байти рівноімовірні.
Всі байти незалежні.
Послідовність однорідна


In [ ]:
#L20       
def l20_gen(t):
    seq = [0] * 8 * t
    for i in range(20):
        seq[i] = np.random.randint(2)
    for j in range(20, 8 * t):
        seq[j] = seq[j - 3] ^ seq[j - 5] ^ seq[j - 9] ^ seq[j - 20]
    return bits_to_bytes(seq)

print("Генератор L20")
test(l20_gen(n))    

Генератор L20
Всі байти рівноімовірні.
Всі байти незалежні.
Послідовність однорідна


In [ ]:
#L89
def l89_gen(t):
    seq = [0] * 8 * t
    for i in range(89):
        seq[i] = np.random.randint(2)
    for j in range(89, 8 * t):
        seq[j] = seq[j - 38] ^ seq[j - 89]
    return bits_to_bytes(seq)

print("Генератор L89")
test(l89_gen(n))    

Генератор L89
Всі байти рівноімовірні.
Всі байти незалежні.
Послідовність однорідна


In [ ]:
#Geffe
def geffe_gen(t):
    
    x = np.random.randint(2, size=(10))
    x = np.concatenate((x, np.zeros(8 * t - 10, dtype=int)))
    y = np.random.randint(2, size=(8))
    y = np.concatenate((y, np.zeros(8 * t - 8, dtype=int)))
    s = np.random.randint(2, size=(9))
    s = np.concatenate((s, np.zeros(8 * t - 9, dtype=int)))
    z = np.zeros(8 * t)
    for i in range(11, 8 * t):
        x[i] = (x[i - 11] ^ x[i - 9])
    for i in range(9, 8 * t):
        y[i] = (y[i - 9] ^ y[i - 8] ^ y[i - 7] ^ y[i - 6])
    for i in range(10, 8 * t):
        s[i] = (s[i - 10] ^ s[i - 7])
    z = s & x ^ (1 ^ s) & y
    return bits_to_bytes(z)

print("Генератор Джиффі")
test(geffe_gen(n))    

Генератор Джиффі
Всі байти рівноімовірні.
Всі байти незалежні.
Послідовність однорідна


In [ ]:
#Wolfram
def wolfram_gen(t):
    r = np.zeros(t, dtype=int)
    r[0] = random.randint(1, (2 ** 32) - 1)
    seq = np.zeros(t)
    seq[0] = r[0] % 2
    for i in range(t):
        r[i] = (cyclic_left_shift(r[i - 1], 1, 32) ^ (r[i - 1] | (cyclic_right_shift(r[i], 1, 32))))
        seq[i] = r[i] % 2
    return seq

print("Генератор Вольфрама")
test(wolfram_gen(n))  

Генератор Вольфрама
Всі байти рівноімовірні.
Всі байти незалежні.
Послідовність однорідна


In [ ]:
#Бібліотекар
def librarian_gen(n, name_f):
    f = open(name_f, 'r', encoding='utf-8')
    text = f.read()
    f.close()
    seq_b = bytes(text, 'utf-8')
    seq = np.zeros(n)
    for i in range(n):
        seq[i] = seq_b[i]
    #for s in seq:
    #    print(s)
    return seq

print("Генератор Бібліотекар")
test(librarian_gen(n, "Rocky Horror Hawkins.txt"))  

Генератор Бібліотекар
Всі байти рівноімовірні.
Всі байти незалежні.
Послідовність однорідна


In [72]:
#BM
def BM_gen(t):
    seq = np.zeros(t)
    p = 0xCEA42B987C44FA642D80AD9F51F10457690DEF10C83D0BC1BCEE12FC3B6093E3
    a = 0x5B88C41246790891C095E2878880342E88C79974303BD0400B090FE38A688356
    T_0 = random.randint(0, p - 1)
    seq[0] = T_0
    for i in range(1, t):
        T_i = pow(a, T_0, p)
        seq[i] = T_i
        T_0 = T_i
    print(1)
    return seq

print("Генератор Блюма-Мікалі")
test(BM_gen(n))  

Генератор Блюма-Мікалі
1


KeyError: 2.438235082348006e+76

In [12]:
#BBS 
def BBS_gen(t):
    seq = np.zeros(t)
    p = 0xD5BBB96D30086EC484EBA3D7F9CAEB07
    q = 0x425D2B9BFDB25B9CF6C416CC6E37B59C1F
    n = p * q
    r_0 = random.randint(2, n)
    for i in range(t):
        r_i = pow(r_0, 2, n)
        seq[i] = r_i % 256
        r_0 = r_i 
    return seq 

print("Генератор Блюм-Блюма-Шуба")
test(BBS_gen(n))  

Генератор Блюм-Блюма-Шуба
Всі байти рівноімовірні.
Всі байти незалежні.
Послідовність однорідна


In [ ]:
p = int(0xCEA42B987C44FA642D80AD9F51F10457690DEF10C83D0BC1BCEE12FC3B6093E3)
print(p)
print(random.randint(0, p - 1))

93466510612868436543809057926265637055082661966786875228460721852868821292003
35145321545597973450803825903772685215011388900359849200685246097149295746285


In [55]:

print("Генератор Лемера low")
test(lemer_gen("low", n))

print("Генератор Лемера low")
test(lemer_gen("low", n))

[  9 189  93 ... 180 202 102]
500000


Всі байти рівноімовірні.
Всі байти незалежні.
Послідовність однорідна
